<a href="https://colab.research.google.com/github/math77/ml-classification-basic/blob/main/ml_intro_casadocodigo_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cap. 01

In [ ]:
"""
100% success rate practically impossible to achieve in real life. This algorithm
is very easy to be wrong.
"""

from sklearn.naive_bayes import MultinomialNB

pig1 = [1, 1, 0]
pig2 = [1, 1, 0]
pig3 = [1, 1, 0]
dog1 = [1, 1, 1]
dog2 = [0, 1, 1]
dog3 = [0, 1, 1]

data = [pig1, pig2, pig3, dog1, dog2, dog3]

marks = [1, 1, 1, -1, -1, -1]

test = [[1, 1, 1], [1, 0, 0], [0, 0, 1]]
marks_test = [-1, 1, -1]

model = MultinomialNB()

# Adapt the model to our data and markings
model.fit(data, marks)
result = model.predict(test)

difference = result - marks_test

matching = [d for d in difference if d == 0]

matching_rate = 100.0 * len(matching) / len(test)

print(result)
print(matching_rate)


[-1  1 -1]
100.0


# Cap. 02


In [ ]:
import csv

from sklearn.naive_bayes import MultinomialNB

def calc_percent(result, X, Y):
  difference = result - Y
  matching = [d for d in difference if d == 0]
  
  return 100.0 * len(matching) / len(X)


def data_load():
  X = []
  Y = []

  file_read = open('/content/acesso.csv', 'r')
  reader = csv.reader(file_read)

  # Ignore the header by skipping to the next line.
  next(reader)

  for acessou_home, acessou_como_funciona, acessou_contato, comprou in reader:
    X.append([int(acessou_home), int(acessou_como_funciona), int(acessou_contato)])
    Y.append(int(comprou))
  
  return X, Y


X, Y = data_load()
X_train = X[:90]
Y_train = Y[:90]

X_test = X[-9:]
Y_test = Y[-9:]

model = MultinomialNB()
model.fit(X_train, Y_train)

result = model.predict(X_test)
print(calc_percent(result, X_test, Y_test))
print(len(X_test))

88.88888888888889
9


# Cap. 03

In [ ]:
from sklearn.naive_bayes import MultinomialNB
import pandas as pd


def calc_percent(result, X, Y):
  difference = result - Y
  matching = [d for d in difference if d == 0]

  return 100.0 * len(matching) / len(X)


#df -> dataframe
df = pd.read_csv('/content/buscas.csv')
X_df = df[['home', 'busca', 'logado']]
Y_df = df['comprou']
Xdummies_df = pd.get_dummies(X_df)
Ydummies_df = Y_df

X = Xdummies_df.values
Y = Ydummies_df.values

train_size = int(0.9 * len(Y))
test_size = len(Y) - train_size

train_data = X[:train_size]
train_marks = Y[:train_size]

test_data = X[-test_size:]
test_marks = Y[-test_size:]

model = MultinomialNB()
model.fit(train_data, train_marks)

result = model.predict(test_data)

print(calc_percent(result, test_data, test_marks))

82.0


# Cap. 04

In [ ]:
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
import pandas as pd


df = pd.read_csv('/content/buscas2.csv')
X_df = df[['home', 'busca', 'logado']]
Y_df = df['comprou']

Xdummies_df = pd.get_dummies(X_df)
Ydummies_df = Y_df

X = Xdummies_df.values
Y = Ydummies_df.values

train_percent = 0.9

train_size = int(train_percent * len(Y))
test_size = len(Y) - train_size

train_data = X[:train_size]
train_marks = Y[:train_size]

test_data = X[-test_size:]
test_marks = Y[-test_size:]

model = MultinomialNB()

model.fit(train_data, train_marks)

result = model.predict(test_data)

matchings = result == test_marks

total_matching = sum(matchings)
total_elements = len(test_data)

matching_rate = 100.0 * total_matching / total_elements

matching_base = max(Counter(test_marks).values())
matching_base_rate = 100.0 * matching_base / len(test_marks)

print(matching_rate)
print(matching_base_rate)

82.0
82.0


# Cap. 06

In [ ]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier

def fit_and_predict(algorithm, model, train_data, train_marks, test_data, test_marks):
  model.fit(train_data, train_marks)

  result = model.predict(test_data)
  matchings = result == test_marks
  matching_rate = 100.0 * sum(matchings) / len(test_data)

  print(f"{algorithm} matching rate: {matching_rate}")

  return matching_rate

def real_test(algorithm, model, validation_data, validation_marks):
  result = model.predict(validation_data)
  matchings = result == validation_marks

  matching_rate = 100.0 * sum(matchings) / len(validation_marks)

  print(f"{algorithm} - winner! matching rate: {matching_rate}")



df = pd.read_csv('/content/buscas.csv')
X_df = df[['home', 'busca', 'logado']]
Y_df = df['comprou']

X_dummies = pd.get_dummies(X_df)
Y_dummies = Y_df

X = X_dummies.values
Y = Y_dummies.values

train_percent = 0.8
test_percent = 0.1

train_size = int(train_percent * len(Y))
test_size = int(test_percent * len(Y))
valiadtion_size = len(Y) - train_size - test_size

end_train = train_size + test_size

train_data = X[:train_size]
train_marks = Y[:train_size]

test_data = X[train_size:end_train]
test_marks = Y[train_size:end_train]

validation_data = X[end_train:]
validation_marks = Y[end_train:]

model_nb = MultinomialNB()
model_ada = AdaBoostClassifier()

r_multinomial = fit_and_predict("MultinomialNB", model_nb, train_data, train_marks, test_data, test_marks)
r_ada = fit_and_predict("AdaBoost", model_ada, train_data, train_marks, test_data, test_marks)

winner = None
winner_name = None

if r_multinomial > r_ada:
  winner = model_nb
  winner_name = "MultinomialNB"
else:
  winner = model_ada
  winner_name = "AdaBoost"

real_test(winner_name, winner, validation_data, validation_marks)

MultinomialNB matching rate: 82.0
AdaBoost matching rate: 84.0
AdaBoost - winner! matching rate: 85.0


# Cap. 07

In [7]:
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import LinearSVC
import pandas as pd



def fit_and_predict(algorithm, model, train_data, train_marks, test_data, test_marks):
  model.fit(train_data, train_marks)
  result = model.predict(test_data)

  matchings = result == test_marks

  matching_rate = 100.0 * sum(matchings) / len(test_data)

  print(f"{algorithm} matching rate: {matching_rate}")

  return matching_rate


def real_test(model, validation_data, validation_marks):
  result = model.predict(validation_data)

  matchings = result == validation_marks

  matching_rate = 100.0 * sum(matchings) / len(validation_marks)

  print(f"Winner - matching rate: {matching_rate}")


df = pd.read_csv('/content/situacao_cliente.csv')
X_df = df[['recencia', 'frequencia', 'semanas_de_inscricao']]
Y_df = df['situacao']

Xdummies_df = pd.get_dummies(X_df)
Ydummies_df = Y_df

X = Xdummies_df.values
Y = Ydummies_df.values

train_percent = 0.8
test_percent = 0.1

train_size = int(train_percent * len(Y))
test_size = int(test_percent * len(Y))
validation_size = len(Y) - train_size - test_size

train_data = X[:train_size]
train_marks = Y[:train_size]

end_train = train_size + test_size

test_data = X[train_size:end_train]
test_marks = Y[train_size:end_train]

validation_data = X[end_train:]
validation_marks = Y[end_train:]

results = {}

model_nb = MultinomialNB()
model_ada = AdaBoostClassifier()
model_one_rest = OneVsRestClassifier(LinearSVC(random_state=0, dual=False))
model_one_one = OneVsOneClassifier(LinearSVC(random_state=0, dual=False))

nb_result = fit_and_predict("MultinomialNB", model_nb, train_data, train_marks, test_data, test_marks)
ada_result = fit_and_predict("AdaBoost", model_ada, train_data, train_marks, test_data, test_marks)
one_rest_result = fit_and_predict("OneVsRest", model_one_rest, train_data, train_marks, test_data, test_marks)
one_one_result = fit_and_predict("OneVsOne", model_one_one, train_data, train_marks, test_data, test_marks)

results[nb_result] = model_nb
results[ada_result] = model_ada
results[one_rest_result] = model_one_rest
results[one_one_result] = model_one_one

winner = results[max(results)]

real_test(winner, validation_data, validation_marks)

MultinomialNB matching rate: 72.72727272727273
AdaBoost matching rate: 68.18181818181819
OneVsRest matching rate: 90.9090909090909
OneVsOne matching rate: 100.0
Winner - matching rate: 100.0


# Cap. 08

In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np


def fit_and_predict(algorithm, model, train_data, train_marks):
  k = 10
  
  scores = cross_val_score(model, train_data, train_marks, cv=k)
  matching_rate = np.mean(scores)
  
  print(f"{algorithm} matching rate: {matching_rate}")

  return matching_rate


def real_test(model, validation_data, validation_marks):
  result = model.predict(validation_data)

  matchings = result == validation_marks

  matching_rate = 100.0 * sum(matchings) / len(validation_data)

  print(f"Winner - matching rate: {matching_rate}")


df = pd.read_csv('/content/situacao_cliente.csv')
X_df = df[['recencia', 'frequencia', 'semanas_de_inscricao']]
Y_df = df['situacao']

Xdummies_df = pd.get_dummies(X_df)
Ydummies_df = Y_df

X = Xdummies_df.values
Y = Ydummies_df.values

train_percent = 0.8

train_size = int(train_percent * len(Y))
#validation_size = len(Y) - train_size

train_data = X[:train_size]
train_marks = Y[:train_size]

validation_data = X[train_size:]
validation_marks = Y[train_size:]

results = {}

model_one_rest = OneVsRestClassifier(LinearSVC(random_state=0, dual=False))

model_one_one = OneVsOneClassifier(LinearSVC(random_state=0, dual=False))
model_nb = MultinomialNB()
model_ada = AdaBoostClassifier()

result_one_rest = fit_and_predict("OneVsRest", model_one_rest, train_data, train_marks)
result_one_one = fit_and_predict("OneVsOne", model_one_one, train_data, train_marks)
result_nb = fit_and_predict("MultinomialNB", model_nb, train_data, train_marks)
result_ada = fit_and_predict("AdaBoost", model_ada, train_data, train_marks)

results[result_one_rest] = model_one_rest
results[result_one_one] = model_one_one
results[result_nb] = model_nb
results[result_ada] = model_ada

winner = results[max(results)]
winner.fit(train_data, train_marks)

real_test(winner, validation_data, validation_marks)

OneVsRest matching rate: 0.9166666666666666
OneVsOne matching rate: 0.9888888888888889
MultinomialNB matching rate: 0.8277777777777778
AdaBoost matching rate: 0.7611111111111111
Winner - matching rate: 100.0
